<a href="https://colab.research.google.com/github/monjurkuet/yt-crawler/blob/main/collab_files_to_mysqloverssh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sshtunnel mysql-connector-python paramiko==3.4.0

In [44]:
import os

path = '/content/drive/MyDrive/AI/transcripts/ytdirect'
files=[]
# Check if the directory exists to provide a more informative message
if not os.path.exists(path):
    print(f"The directory '{path}' does not exist. Please check the path and try again.")
else:
    json_files = [f for f in os.listdir(path) if f.endswith('.json')]
    if json_files:
        print(f"JSON files in '{path}':")
        for f in json_files:
            filepath=os.path.join(path, f)
            files.append({'filepath':filepath,'filename':f})
    else:
        print(f"No JSON files found in '{path}'.")

JSON files in '/content/drive/MyDrive/AI/transcripts/ytdirect':


In [45]:
#files[0]

In [33]:
import os
from google.colab import userdata # Keep for direct Colab execution, but prefer os.environ

class ConfigManager:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(ConfigManager, cls).__new__(cls)
            cls._instance._load_config()
        return cls._instance

    def _load_config(self):
        # Helper to get config, prioritizing environment variables
        def get_config_value(key):
            return os.environ.get(key) or userdata.get(key)

        # SSH Tunnel & MySQL Database Credentials
        self.SSH_HOST = get_config_value('SSH_HOST')
        self.SSH_USER = get_config_value('SSH_USER')
        self.SSH_PRIVATEKEY_PATH = get_config_value('SSH_PRIVATEKEY_PATH') # Path to private key on Colab
        self.LOCAL_PORT = 3307 # Local port for SSH tunnel
        self.REMOTE_MYSQL_HOST = '127.0.0.1'
        self.REMOTE_MYSQL_PORT = 3306
        self.DATABASE_NAME = get_config_value('DATABASE_NAME')
        self.DATABASE_PASSWORD = get_config_value('DATABASE_PASSWORD')
        self.MYSQL_USER = get_config_value('MYSQL_USER')

    def get_config(self):
        return {
            'SSH_HOST': self.SSH_HOST,
            'SSH_USERNAME': self.SSH_USER,
            'SSH_PRIVATEKEY_PATH': self.SSH_PRIVATEKEY_PATH,
            'LOCAL_PORT': self.LOCAL_PORT,
            'REMOTE_MYSQL_HOST': self.REMOTE_MYSQL_HOST,
            'REMOTE_MYSQL_PORT': self.REMOTE_MYSQL_PORT,
            'DATABASE_NAME': self.DATABASE_NAME,
            'DATABASE_PASSWORD': self.DATABASE_PASSWORD,
            'MYSQL_USER':self.MYSQL_USER
        }

# Example usage (for testing purposes, not part of the module's core function)
if __name__ == '__main__':
    config = ConfigManager()
    print(config.LOCAL_PORT)


3307


In [34]:
from sshtunnel import SSHTunnelForwarder
import mysql.connector

# Get configuration from ConfigManager
cfg = ConfigManager().get_config()

# Initialize SSH tunnel and MySQL connection to None
ssh_tunnel = None
mysql_connection = None

try:
    # Create SSH Tunnel
    ssh_tunnel = SSHTunnelForwarder(
        (cfg['SSH_HOST'], 22),
        ssh_username=cfg['SSH_USERNAME'],
        ssh_private_key=cfg['SSH_PRIVATEKEY_PATH'],
        remote_bind_address=(cfg['REMOTE_MYSQL_HOST'], cfg['REMOTE_MYSQL_PORT'])
    )
    ssh_tunnel.start()
    print("SSH tunnel established")

    # Connect to MySQL through the tunnel
    mysql_connection = mysql.connector.connect(
        host=cfg['REMOTE_MYSQL_HOST'],
        port=ssh_tunnel.local_bind_port,
        user=cfg['MYSQL_USER'],
        password=cfg['DATABASE_PASSWORD'],
        database=cfg['DATABASE_NAME']
    )
    print("MySQL database connection established")

    # At this point, the connection is active and can be used for database operations

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # The finally block will not close the connection immediately as it's needed for subsequent steps
    # The connection and tunnel will be closed at the end of the entire task.
    pass


SSH tunnel established
MySQL database connection established


In [35]:
try:
    # Create a cursor object
    cursor = mysql_connection.cursor()

    # Define the CREATE TABLE SQL statement
    create_table_query = """
    CREATE TABLE `transcript_files` (
          `id` INT NOT NULL AUTO_INCREMENT,
          `filename` VARCHAR(255) NOT NULL COLLATE 'utf8mb4_0900_ai_ci',
          `filepath` VARCHAR(255) NOT NULL COLLATE 'utf8mb4_0900_ai_ci',
          PRIMARY KEY (`id`) USING BTREE,
          UNIQUE INDEX `filename` (`filename`) USING BTREE
        )
        COLLATE='utf8mb4_0900_ai_ci'
        ENGINE=InnoDB
        ;
    """

    # Execute the CREATE TABLE statement
    cursor.execute(create_table_query)
    mysql_connection.commit()
    print("Table 'transcript_files' created successfully or already exists.")

except Exception as e:
    print(f"Error creating table: {e}")
finally:
    # Close the cursor
    if 'cursor' in locals() and cursor is not None:
        cursor.close()

Table 'transcript_files' created successfully or already exists.


In [46]:
try:
    cursor = mysql_connection.cursor()

    # Iterate through the list of filenames and insert them into the table
    print('Iterating through the list of files and inserting them into the table......')
    for file_ in files:
        # Construct the INSERT statement
        insert_query = "INSERT INTO transcript_files (filename, filepath) VALUES (%s, %s)"
        # Execute the INSERT statement
        filename=file_['filename']
        filepath=file_['filepath']
        cursor.execute(insert_query, (filename,filepath))

    # Commit the changes to the database
    mysql_connection.commit()
    print(f"{len(json_files)} data inserted into 'transcript_files' table.")

except Exception as e:
    print(f"Error inserting filenames: {e} : filename: {filename}")
finally:
    if 'cursor' in locals() and cursor is not None:
        cursor.close()


Iterating through the list of filenames and inserting them into the table......
1181 data inserted into 'transcript_files' table.
